In [1]:
import os
import sys
from anthropic import Anthropic, AnthropicError

notebook_path = os.getcwd()
project_path = os.path.abspath(os.path.join(notebook_path, '..','..'))
print(f"Project path: {project_path}")
# Add the project path to sys.path
sys.path.append(project_path)

Project path: c:\Users\dalej\Documents\DragonRegen


In [3]:
from src.DataCreator.ColGenerators.ColGenRegistry import ColGenResistry

ColGenResistry.get_all_descriptions()


{'ColBasic': 'A back stop column for your basic data types with no other identifying attributes.',
 'StringBasic': 'This is a basic string column generator that can be used to generate a column of strings with a specified length and format. It is not intended for free text or complex string generation.',
 'Categorical': 'a column that represents a category or type, often with a limited set of values.',
 'StringFirstName': 'A column that stores the first name of a person.',
 'StringLastName': 'A column that stores the Last name of a person.'}

In [8]:
import json

s_st = """--> The Beginning of the ColGenResistry.get_all_descriptions() output
"""+ '\n'.join([str(key)+': '+str(val) for key, val in ColGenResistry.get_all_descriptions().items()])+"""
--> THE END
"""
               
print(s_st)

--> The Beginning of the ColGenResistry.get_all_descriptions() output
ColBasic: A back stop column for your basic data types with no other identifying attributes.
StringBasic: This is a basic string column generator that can be used to generate a column of strings with a specified length and format. It is not intended for free text or complex string generation.
Categorical: a column that represents a category or type, often with a limited set of values.
StringFirstName: A column that stores the first name of a person.
StringLastName: A column that stores the Last name of a person.
--> THE END



In [5]:
o_colgen = ColGenResistry.get_col_generator('ColBasic')
print(o_colgen)
print(o_colgen.get_metadata_json())
print(o_colgen.get_examples())

<class 'src.DataCreator.ColGenerators.ColBasic.ColBasic'>
{'description': 'Place the description of the column here.', 'unique_fl': True, 'default_value': None}
Example 1:
        Purpose: "This table is used to store user information."
        Column Info: "user_id": "unique ID representing each user."
        Output:
        <JSON_Template>
        {"name": "user_id", "type": "Integer", "nullable": False, 
            "metadata": {"description": "unique ID representing each user.", 
            "unique_fl": True,
            "default_value": None}}
        </JSON_Template>


### Data Column Type

In [2]:
from src.AIGuardian.ActionsFrame.DataColumnType import DataColumnType

# Inputs
action_parameters = {
                "table_name": 'members',
                "purpose": "To track all the information about members of the gym.",
                "column_name": 'membership',
                "description": 'The kind of membership the member has.',
            }

test_type_action = DataColumnType(input_params=action_parameters)
print(test_type_action)

In [3]:
test_type_action.run(user_prompt="")
print(test_type_action.response)

create_params: {'max_tokens': 10000, 'temperature': 0.1, 'stop_sequences': ['</JSON_Template>'], 'model': 'claude-3-7-sonnet-20250219', 'messages': [{'role': 'user', 'content': '\nRespond in JSON format like this:\n{\n    "choice": "selected_option",\n    "reason": "brief explanation"\n}\n\n<Examples>\nExample 1:\n<column_information>\nTable Purpose: Stores information about gym members.\nColumn Info: "member_name": "The first name of the gym member."\n</column_information>\nOutput:\n<JSON_Template>\n{\n    "choice": "First_Name",\n    "reason": "This represents the first name of a person, which is a common attribute in member records."\n}\n</JSON_Template>\n\nExample 2:\n<column_information>\nTable Purpose: Stores information about gym members.\nColumn Info: "subscription": "The type of subscription the member has."\n</column_information>\nOutput:\n<JSON_Template>\n{{\n    "choice": "Categorical",\n    "reason": "There is a finite number of options for memberships at a gym."\n}}\n</JS

In [4]:
print(test_type_action.output_params)

{'reason': 'There is a finite number of options for memberships at a gym.', 'col_type': 'Categorical'}


### Data Column Refiner

In [5]:
from src.AIGuardian.ActionsFrame.DataColumnRefiner import DataColumnRefiner

# Inputs
action_parameters = {
                "table_name": 'members',
                "purpose": "To track all the information about members of the gym.",
                "column_name": 'membership',
                "description": 'The kind of membership the member has.',
            }

test_action = DataColumnRefiner(input_params=action_parameters, parent_action=test_type_action)
print(test_action)

In [6]:
test_action.run(user_prompt="")
print(test_action.response)


create_params: {'max_tokens': 10000, 'temperature': 0.1, 'stop_sequences': ['</JSON_Template>'], 'model': 'claude-3-7-sonnet-20250219', 'messages': [{'role': 'user', 'content': 'Please take the following <column_information> to fill in JSON values and add additional metadata for the column.\n        <column_information>\n        Table Purpose: To track all the information about members of the gym.\n        Column Info: "membership": "The kind of membership the member has."\n        </column_information>\n        Please provide the response in a JSON format like the <JSON_Template> below. If the JSON value has (Optional) in the value you may or may not included.\n        There are examples below at <Examples>.\n        <JSON_Template>\n        {"name": "example_column", "type": "Integer", "nullable": true, "metadata": {"description": "Place the description of the column here. Add detail if current description is vague.", "unique_fl": "Choose True or False", "default_value": "(Optional) 

## Test Data Struct

In [ ]:
# from src.AIGuardian.ActionsFrame.DataStructCreate import DataStructCreate

# datastruct_action = DataStructCreate()
# prompt = """Create a data structure for a gym membership system."""
# datastruct_action.run(prompt)
# print(datastruct_action.response)

SQL: INSERT INTO ai_actions_log (action_name, action_version, parent_action_id, group_action_id, description, sequence_number, created_dt, updated_dt, status, metadata) 
        OUTPUT INSERTED.action_id
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        
create_params: {'max_tokens': 10000, 'temperature': 0.1, 'stop_sequences': ['</JSON_Template>'], 'model': 'claude-3-7-sonnet-20250219', 'messages': [{'role': 'user', 'content': 'Please take the following <business_requirements> and create the table list and schema for the the request.\n        <business_requirements>\n        Create a data structure for a gym membership system.\n        </business_requirements>\n        Please provide the response in a JSON format like the <JSON_Template> below:\n        <JSON_Template>\n        {"table_name_1": {\n            "purpose": "short description of tables usage in the architecture",\n            "fields":{"column_name_1": "unique ID, short description of column",\n            , "column_n